# Assignment 4
## Understaning scaling of linear algebra operations on Apache Spark using Apache SystemML

In this assignment we want you to understand how to scale linear algebra operations from a single machine to multiple machines, memory and CPU cores using Apache SystemML. Therefore we want you to understand how to migrate from a numpy program to a SystemML DML program. Don't worry. We will give you a lot of hints. Finally, you won't need this knowledge anyways if you are sticking to Keras only, but once you go beyond that point you'll be happy to see what's going on behind the scenes. Please make sure you run this notebook from an Apache Spark 2.3 notebook.

So the first thing we need to ensure is that we are on the latest version of SystemML, which is 1.2.0 (as of Feb. '19)
Please use the code block below to check if you are already on 1.2.0 or higher.

In [2]:
from systemml import MLContext
ml = MLContext(spark)
ml.version()

'1.2.0'

If you are blow version 1.2.0 please execute the next two code blocks

In [3]:
!pip install systemml

  Using cached https://files.pythonhosted.org/packages/e3/18/4f013c3c3051f4e0ffbaa4bf247050d6d5e527fe9cb1907f5975b172f23f/numpy-1.16.2-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/38/7f/7d21bd89a97603a14d3770ca912d4402c394a8e50642f9b8b5cc6867a61a/scikit_learn-0.20.3-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f0/30/526bee2ce18c066f9ff13ba89603f6c2b96c9fd406b57a21a7ba14bf5679/scipy-1.2.1-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/74/24/0cdbf8907e1e3bc5a8da03345c23cbed7044330bb8f73bb12e711a640a00/pandas-0.24.2-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/de/62/6358ccd27b9e340affaa29e3e678b5d93a4b17f6bd7533fd6e6e87930b12/Pillow-6.0.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/41/17/c62faccbfbd163c7f57f3844689e3a78bae1f403648a6afb1d0866d87fbb/python_dateutil-2.8.0-py2.py3-n

Now we need to create two sym links that the newest version is picket up - this is a workaround and will be removed soon

In [4]:
!ln -s -f ~/user-libs/python3/systemml/systemml-java/systemml-1.2.0-extra.jar ~/user-libs/spark2/systemml-1.2.0-extra.jar
!ln -s -f ~/user-libs/python3/systemml/systemml-java/systemml-1.2.0.jar ~/user-libs/spark2/systemml-1.2.0.jar

Now please restart the kernel and make sure the version is correct

In [5]:
from systemml import MLContext
ml = MLContext(spark)
ml.version()

'1.2.0'

Congratulations, if you see version 1.2.0 or higher, please continue with the notebook...

In [6]:
from systemml import MLContext, dml
import numpy as np
import time

Then we create an MLContext to interface with Apache SystemML. Note that we pass a SparkSession object as parameter so SystemML now knows how to talk to the Apache Spark cluster

In [7]:
ml = MLContext(spark)

Now we create some large random matrices to have numpy and SystemML crunch on it

In [8]:
u = np.random.rand(1000,10000)
s = np.random.rand(10000,1000)
w = np.random.rand(1000,1000)

Now we implement a short one-liner to define a very simple linear algebra operation

In case you are unfamiliar with matrxi-matrix multiplication: https://en.wikipedia.org/wiki/Matrix_multiplication

sum(U' * (W . (U * S)))


| Legend        |            |   
| ------------- |-------------| 
| '      | transpose of a matrix | 
| * | matrix-matrix multiplication      |  
| . | scalar multiplication      |   



In [9]:
start = time.time()
res = np.sum(u.T.dot(w * u.dot(s)))
print (time.time()-start)

1.8185393810272217


As you can see this executes perfectly fine. Note that this is even a very efficient execution because numpy uses a C/C++ backend which is known for it's performance. But what happens if U, S or W get such big that the available main memory cannot cope with it? Let's give it a try:

In [10]:
#u = np.random.rand(10000,100000)
#s = np.random.rand(100000,10000)
#w = np.random.rand(10000,10000)

After a short while you should see a memory error. This is because the operating system process was not able to allocate enough memory for storing the numpy array on the heap. Now it's time to re-implement the very same operations as DML in SystemML, and this is your task. Just replace all ###your_code_goes_here sections with proper code, please consider the following table which contains all DML syntax you need:

| Syntax        |            |   
| ------------- |-------------| 
| t(M)      | transpose of a matrix, where M is the matrix | 
| %*% | matrix-matrix multiplication      |  
| * | scalar multiplication      |   

## Task

In order to show you the advantage of SystemML over numpy we've blown up the sizes of the matrices. Unfortunately, on a 1-2 worker Spark cluster it takes quite some time to complete. Therefore we've stripped down the example to smaller matrices below, but we've kept the code, just in case you are curious to check it out. But you might want to use some more workers which you easily can configure in the environment settings of the project within Watson Studio. Just be aware that you're currently limited to free 50 capacity unit hours per month wich are consumed by the additional workers.

In [11]:
script = """
U = rand(rows=1000,cols=10000)
S = rand(rows=10000,cols=1000)
W = rand(rows=1000,cols=1000)
res = sum(t(U) %*% (W * (U %*% S)))
"""

To get consistent results we switch from a random matrix initialization to something deterministic

In [12]:
#prog = dml(script).input('U', u).input('S', s).input('W', w).output('res')
prog = dml(script).output('res')
res = ml.execute(prog).get('res')
print(res)

SystemML Statistics:
Total execution time:		4.216 sec.
Number of executed Spark inst:	0.


6254913692715.805


If everything runs fine you should get *6244089899151.321* as result. Feel free to submit your DML script to the grader now!

### Submission

In [6]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2019-03-01 21:34:42--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: 'rklib.py'

rklib.py            100%[===================>]   2.24K  --.-KB/s    in 0s      

2019-03-01 21:34:42 (54.5 MB/s) - 'rklib.py' saved [2289/2289]



In [7]:
from rklib import submit
key = "esRk7vn-Eeej-BLTuYzd0g"


email = "###_YOUR_CODE_GOES_HERE_###"

In [8]:
part = "fUxc8"
secret = "###_YOUR_CODE_GOES_HERE_###"
submit(email, secret, key, part, [part], script)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"P1p3F","id":"tE4j0qhMEeecqgpT6QjMdA~P1p3F~2R7WWzxpEemzJAqGvaN4YA","courseId":"tE4j0qhMEeecqgpT6QjMdA"}],"paging":{},"linked":{}}
-------------------------
